# Algo DRL SAC/IQL — Colab End-to-End

This notebook sets up a GPU environment on Colab, installs dependencies, clones your repo, and runs the full pipeline: data download → offline IQL pretrain → SAC fine-tune → evaluation.

In [ ]:
# Check GPU availability
!nvidia-smi || true

In [ ]:
# Install core dependencies
# Colab typically supports CUDA 12.1 wheels; if it fails, fallback to default CPU wheels.
%%bash
pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 \n+      || pip -q install torch torchvision torchaudio
pip -q install d3rlpy==2.8.1 gymnasium pandas numpy plotly vectorbt requests pyyaml

## Clone your repository (always pulls latest)
Edit `REPO_URL` and (optionally) `BRANCH`. The notebook will clone if missing, or pull to update.

In [ ]:
import os, subprocess, sys, pathlib
REPO_URL = os.environ.get('REPO_URL', 'https://github.com/your-account/algo-drl-sac-iql.git')
BRANCH   = os.environ.get('REPO_BRANCH', 'main')
TARGET   = pathlib.Path('/content/algo-drl-sac-iql')

if not TARGET.exists():
    !git clone -b $BRANCH $REPO_URL $TARGET
else:
    %cd $TARGET
    !git fetch origin $BRANCH && git checkout $BRANCH && git pull
    %cd /content

%cd $TARGET

## Quick knobs (shorter runs in Colab)

In [ ]:
import os
os.environ['QA_STEPS'] = os.environ.get('QA_STEPS', '5000')
os.environ['QA_DEVICE'] = os.environ.get('QA_DEVICE', 'cuda')
os.environ['CONFIG'] = os.environ.get('CONFIG', 'config/config.yaml')
print('QA_STEPS =', os.environ['QA_STEPS'])
print('QA_DEVICE=', os.environ['QA_DEVICE'])
print('CONFIG  =', os.environ['CONFIG'])

## 1) Download OHLCV (with volume fields)

In [ ]:
!python scripts/download_ohlcv_binance.py \n+    --symbol BTCUSDT --interval 3600 \n+    --start '2024-06-10' --end '2025-10-16' \n+    --output-dir data

## 2) Offline pretrain (IQL)

In [ ]:
!python -m src.run_offline_pretrain --config config/config.yaml --device cuda

## 3) Online fine-tune (SAC)

In [ ]:
!python -m src.run_sac_finetune --config config/config.yaml --device cuda

## 4) Evaluation (test window with warm-up)

In [ ]:
!python -m src.run_walkforward --config config/config.yaml --device cuda

## View charts in notebook

In [ ]:
from IPython.display import IFrame, display
display(IFrame('evaluation/charts/equity_BTCUSDT.html', width=1000, height=600))
display(IFrame('evaluation/charts/candlestick_BTCUSDT_SepOct2025.html', width=1000, height=600))